In [1]:
import re
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../../Data/Raw/some_cmd_v3_03062022.csv", sep=";")
print(df.shape)
print(len(df[df['full_text'].isna()]), "nan texts")
df.drop_duplicates()
df.shape, len(df[df['text3'].isna()])

(16253, 50)
0 nan texts


((16253, 50), 11367)

In [3]:
for ir, row in df.iterrows():
    start_1 = row["full_text"].find(row["text1"])
    start_2 = row["full_text"].find(row["text2"], start_1 + len(row["text1"]))    
    assert start_1 >= 0
    assert start_2 >= start_1
    if type(row['text3']) != float:
        start_3 = row["full_text"].find(row["text3"], start_2 + len(row["text2"]))
        assert start_3 >= start_2

In [4]:
# сплит сгенерированной части на трейн и валид 
validIndex = set()
for c in ['y_1', 'action_1', 'y_2', 'action_2']:
    #if c in ["full_text", "text1", "text2", "text3", "y_name_1", "y_name_2", "y_name_3", "type"]:
#        continue
    train_x, valid_x = train_test_split(df.index, test_size=0.03, random_state=69, shuffle=True, stratify=df[c])
    validIndex |= set(valid_x)
print(len(validIndex) / df.shape[0])

subsets = ["valid" if i in validIndex else "train" for i in range(df.shape[0])]
df["subset"] = subsets
df.loc[df["type"].isin(['students', 'crowdsource']),"subset"] = "test"

0.1152402633359995


In [5]:
for c in df.columns:
    if c in ["full_text", "text1", "text2", "text3", "y_name_1", "y_name_2", "y_name_3", "type", "subset"]:
        continue
    trains_counts = df.loc[df["subset"]=="train", c].value_counts().to_dict()
    valid_counts = df.loc[df["subset"]=="valid", c].value_counts().to_dict()
    test_counts = df.loc[df["subset"]=="test", c].value_counts().to_dict()
    print(c)
    for k in trains_counts:
        if k in test_counts:
            print(k, trains_counts[k], valid_counts[k], test_counts[k], valid_counts[k] / trains_counts[k], test_counts[k] / trains_counts[k])
        else:
            print(k, trains_counts[k], valid_counts[k], valid_counts[k] / trains_counts[k])
    print()

y_1
5 5612 731 0.13025659301496792
8 1193 144 0.12070410729253982
6 1138 151 0.13268892794376097
7 765 91 0.11895424836601307
3 746 93 0.12466487935656836
9 729 82 0.11248285322359397
11 722 97 0.13434903047091412
1 717 106 0.14783821478382148
2 703 91 0.12944523470839261
0 694 107 0.15417867435158503
4 687 94 0.13682678311499272
10 674 86 0.12759643916913946

action_1
7 5800 749 0.12913793103448276
6 1929 248 0.128564022809746
5 1433 187 0.13049546406140963
0 956 120 0.12552301255230125
8 818 111 0.1356968215158924
9 785 85 0.10828025477707007
10 772 107 0.13860103626943004
4 703 91 0.12944523470839261
3 387 58 0.14987080103359174
1 336 51 0.15178571428571427
2 330 48 0.14545454545454545
11 86 11 0.12790697674418605
12 45 7 0.15555555555555556

direction_1
0 11527 1489 0.12917498048061074
3 822 118 0.1435523114355231
1 392 42 0.10714285714285714
5 329 36 0.1094224924012158
4 319 55 0.1724137931034483
8 315 41 0.13015873015873017
6 302 45 0.1490066225165563
7 280 37 0.13214285714285715

KeyError: 5

In [6]:
# в случае изменения функции, изменить её и в обработке коротких команд в Preprocess_for_classifier.ipynb
#punct_characters = re.escape(".,!?:;-#$*()[]'\"/|^%~`_+-")
def removePunctETC(value):
    if type(value) is float:
        return value
    value = value.lower()
    value = re.sub("[^\w\s°]", " ", value)
    value = re.sub(" {2,}", " ", value)
    return value

df["full_text"] = df["full_text"].map(removePunctETC)
df["text1"] = df["text1"].map(removePunctETC)
df["text2"] = df["text2"].map(removePunctETC)
df["text3"] = df["text3"].map(removePunctETC)

df.shape

(16253, 51)

In [7]:
df.to_csv("../../Data/Interim/some_cmd_v3_03062022_low_split.csv", index=False)